# CP1: Data Wrangling

Include answers to these questions in your submission:

1. What kind of cleaning steps did you perform?

2. How did you deal with missing values, if any?

3. Were there outliers, and how did you handle them?

1. Steps I took to clean my data involved deleting irrelevant features, converting dates to datetime objects, and writing functions to clean up urls and hashtags. While I don't necessarily anticipate needing clean hashtags in the future, clean sources and urls could become useful if I were to expand my source list from Adam Schefter to local beat writers for identication purposes.

* Since this is my second attempt at downloading as many Schefter tweets as possible, I still need to go through and manually label each tweet. 

2. There aren't any missing values.

3. I'm not sure it's possible for an outlier to exist in my particular dataset.

In [2]:
# import requirements

import csv
from datetime import datetime
import os
import re
import requests
import twitter
import lxml.html

In [3]:
# specify credentials to access Twitter API

api = twitter.Api(consumer_key='HHFP7OMrBwTxLnuCNA7bFo10E',
                  consumer_secret='rgjhLwNMLsly7lB63obWhDLbUoi5rte9HSWrXQEIocX7ftJqOh',
                  access_token_key='2859784876-xEksUR0FQ6PYzgHzt86L6CVmJn2S88dgvi8EM8g',
                  access_token_secret='2ZTMVXOiyeBpKvdRYkm3eIFIJap915OS5v6ZfihQjPf37')

In [4]:
# specify Twitter username to be scraped

screen_name = 'AdamSchefter'

In [5]:
# call the Twitter API

first_200 = api.GetUserTimeline(screen_name=screen_name, count=200)

In [6]:
# Sample breakdown

print("There are %d tweets in our dataset, which has the type %s." % (len(first_200), type(first_200)))
print()
print("Here's a sample of what the data from one tweet looks like:")
print(first_200[0])
print()
print("Each tweet is stored in a %s data structure." % type(first_200[0]))

There are 200 tweets in our dataset, which has the type <class 'list'>.

Here's a sample of what the data from one tweet looks like:
{"created_at": "Tue Apr 02 13:05:51 +0000 2019", "favorite_count": 820, "hashtags": [], "id": 1113065005405102080, "id_str": "1113065005405102080", "lang": "en", "quoted_status": {"created_at": "Tue Apr 02 13:01:12 +0000 2019", "favorite_count": 1144, "hashtags": [], "id": 1113063833818415104, "id_str": "1113063833818415104", "lang": "en", "retweet_count": 347, "source": "<a href=\"https://about.twitter.com/products/tweetdeck\" rel=\"nofollow\">TweetDeck</a>", "text": "Sources: the Patriots and DE Michael Bennett have agreed to a reworked two-year deal with a base value of $16.75M,\u2026 https://t.co/XakOELAwKc", "truncated": true, "urls": [{"expanded_url": "https://twitter.com/i/web/status/1113063833818415104", "url": "https://t.co/XakOELAwKc"}], "user": {"created_at": "Tue Jul 21 20:57:56 +0000 2009", "description": "NFL Insider for ESPN. Co-host of the

In [7]:
def get_tweets(first_200, screen_name, last_id):
    all_tweets = []
    all_tweets.extend(first_200)
    for i in range(900):
        new = api.GetUserTimeline(screen_name=screen_name, max_id=last_id-1)
        if len(new) > 0:
            all_tweets.extend(new)
            last_id = new[-1].id
        else:
            break
    
    return all_tweets

In [10]:
all_tweets = get_tweets(first_200, screen_name, first_200[-1].id)

In [11]:
# check to see how many tweets we actually have

print("There are %d tweets stored in a list as the all_tweets variable." % len(all_tweets))
print("The most recent tweet in our collection was sent %s and the oldest tweet was sent %s." % (
    all_tweets[0].created_at, 
    all_tweets[-1].created_at)
     )

There are 3222 tweets stored in a list as the all_tweets variable.
The most recent tweet in our collection was sent Tue Apr 02 13:05:51 +0000 2019 and the oldest tweet was sent Tue Aug 07 17:39:19 +0000 2018.


In [12]:
# take a look at an individual observation

print(all_tweets[0])

{"created_at": "Tue Apr 02 13:05:51 +0000 2019", "favorite_count": 820, "hashtags": [], "id": 1113065005405102080, "id_str": "1113065005405102080", "lang": "en", "quoted_status": {"created_at": "Tue Apr 02 13:01:12 +0000 2019", "favorite_count": 1144, "hashtags": [], "id": 1113063833818415104, "id_str": "1113063833818415104", "lang": "en", "retweet_count": 347, "source": "<a href=\"https://about.twitter.com/products/tweetdeck\" rel=\"nofollow\">TweetDeck</a>", "text": "Sources: the Patriots and DE Michael Bennett have agreed to a reworked two-year deal with a base value of $16.75M,\u2026 https://t.co/XakOELAwKc", "truncated": true, "urls": [{"expanded_url": "https://twitter.com/i/web/status/1113063833818415104", "url": "https://t.co/XakOELAwKc"}], "user": {"created_at": "Tue Jul 21 20:57:56 +0000 2009", "description": "NFL Insider for ESPN. Co-host of the Fantasy Focus Football podcast. Graduate of Belmont Hill and Wesleyan.", "favourites_count": 9311, "followers_count": 370045, "frien

In [13]:
# features I'm looking closer at

print("How the data appear:")
print("The created_at attribute:", all_tweets[1].created_at)
print("The hashtags attribute:", all_tweets[1].hashtags)
print("The urls attribute:", all_tweets[1].urls)
print("The source attribute:", all_tweets[1].source)

How the data appear:
The created_at attribute: Tue Apr 02 11:35:48 +0000 2019
The hashtags attribute: []
The urls attribute: [URL(URL=https://t.co/yH6nVvhfy5, ExpandedURL=https://twitter.com/i/web/status/1113042341038886913)]
The source attribute: <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>


In [14]:
# function to convert from string to datetime

def string_to_datetime(date_str):
    """
    Turns a string including date and time like this - Sun Jul 01 21:06:07 +0000 2018 - to a Python datetime object
    like this - datetime.datetime(2018, 7, 1, 21, 6, 7, tzinfo=datetime.timezone.utc)
    """
    return datetime.strptime(date_str, '%a %b %d %H:%M:%S %z %Y')

In [15]:
# functions to clean up urls and sources
# which may be useful if I need to expand beyond schefter

'''
clean_urls turns data with any number of expanded urls like this - 
    [URL(URL=https://t.co/sYCFHKxzBf, ExpandedURL=https://youtu.be/34bFgA3H3hQ)]- to a list like this - 
    ["https://youtu.be/34bFgA3H3hQ"]
    
clean_source turns data including the source and some html like this - 
    <a href="https://www.sprinklr.com" rel="nofollow">Sprinklr</a> - to a list like this -
    ['Sprinklr']
    
clean_hashtags turns data with any number of hashtags like this - [Hashtag(Text='STEMonStation')] - to a list like this -
    ['STEMonStation']
'''


def clean_urls(urls):
    cleaned = []
    if len(urls) >= 1:
        for i in range(len(urls)):
            cleaned.append(urls[i].expanded_url)
    return(cleaned)
        

def clean_source(source):
    raw = lxml.html.document_fromstring(source)
    return raw.cssselect('body')[0].text_content()

def clean_hashtags(hashtags):
    cleaned = []
    if len(hashtags) >= 1:
        for i in range(len(hashtags)):
            cleaned.append(hashtags[i].text)        
    return cleaned

In [16]:
def write_to_csv(tweets, filename):
    # the headers are the fields that we identified to keep
    headers = ['id', 'full_text', 'hashtags', 'urls', 'created_at', 'favorite_count', 'retweet_count', 'source']
    
    # here we create the file and write the header row with the headers list
    # note that the 'filename' argument will be the name of the csv file
    with open(filename + '.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(headers)
        
        # in this loop, we write a new row for each tweet object, with the data taken from the tweet object in 
        # the order we listed the headers
        # note where we call the helper functions from step 4 on hashtags, urls, and source
        for item in tweets:
            writer.writerow([item.id, 
                             item.full_text, 
                             clean_hashtags(item.hashtags), 
                             clean_urls(item.urls), 
                             item.created_at, 
                             item.favorite_count, 
                             item.retweet_count, 
                             clean_source(item.source)])
    csvfile.close()

In [17]:
# now we call the function, passing in the all_tweets list
# here the filename will be the screen_name variable defined in step 2 with "_tweets" after it (e.g. NASA_tweets.csv),
# but you can change it to whatever you want
write_to_csv(all_tweets, screen_name + '_tweets')

# This is the second attempt to retrieve data

It appears there may be a ~3200 tweet limit for scraping a particular user timeline. 

The first attempt I used at collected Schefter's tweets utilized an R script that produced another dataset consisting of another ~3200 rows from May 21, 2018 to February 17, 2019. There are thus overlapping rows between the two datasets that could be merged to create 

This first dataset can be seen here: https://github.com/lelarson/Springboard/blob/master/schefter_tweets_2018.csv

It appears possible to stream Schefter's tweets into a file using Tweepy: https://tweepy.readthedocs.io/en/v3.5.0/ 



